1. Initialize energy system

In [ ]:
from assetra.core import EnergySystem

energy_system = EnergySystem()

: 

2. Add units

In [ ]:
import xarray as xr
import numpy as np

: 

In [ ]:
from assetra.core import (
	DemandUnit,
	StochasticUnit
	)
# add demand
energy_system.add_unit(
	DemandUnit(
		id=0,
		hourly_demand=xr.DataArray(
			data=np.ones(8760),
			coords=dict(
				time=xr.date_range(
					start="2016-01-01 00:00",
					end="2016-12-31 23:00",
					freq="H",
					inclusive="both",
				)
			)
		)
	)
)
	
# add one hundred 1 MW generators
for i in range(100):
	energy_system.add_unit(
		StochasticUnit(
			id=i+1,
			nameplate_capacity=1,
			hourly_capacity=xr.DataArray(
				data=np.ones(8760)*0.01,
				coords=dict(
					time=xr.date_range(
						start="2016-01-01 00:00",
						end="2016-12-31 23:00",
						freq="H",
						inclusive="both",
					)
				)),
			hourly_forced_outage_rate=xr.DataArray(
				data=np.ones(8760)*0.05,
				coords=dict(
					time=xr.date_range(
						start="2016-01-01 00:00",
						end="2016-12-31 23:00",
						freq="H",
						inclusive="both",
					)
				)),
			)
		)

3. Create probabilistic analysis object

In [ ]:
from assetra.probabilistic_analysis import ProbabilisticSimulation

simulation = ProbabilisticSimulation(
    energy_system=energy_system
	start_hour=0,
	end_hour=8760,
	trial_size=100
	)


 4. Quantify resource adequacy

In [ ]:

from assetra.adequacy_metrics import ExpectedUnservedEnergy

# calculate resource adequacy
eue_inst = ExpectedUnservedEnergy(probabilistic_simulation=simulation)
eue = eue_inst.evaluate()

5. Quantify resource contribution of additional generator

In [ ]:
from assetra.contribution_metrics import EffectiveLoadCarryingCapability

# declare additional solar plant
addition = EnergySystem()
addition.add_unit(
	StochasticUnit(
		id=101,
		nameplate_capacity=1,
		hourly_capacity=xr.DataArray(
			data=np.ones(8760)*0.01,
			coords=dict(
				time=xr.date_range(
					start="2016-01-01 00:00",
					end="2016-12-31 23:00",
					freq="H",
					inclusive="both",
				)
			)),
		hourly_forced_outage_rate=xr.DataArray(
			data=np.ones(8760)*0.05,
			coords=dict(
				time=xr.date_range(
					start="2016-01-01 00:00",
					end="2016-12-31 23:00",
					freq="H",
					inclusive="both",
				)
			)),
		)
	)

# calculate resource contribution
elcc_inst = EffectiveLoadCarryingCapability(
	energy_system,
    addition,
    probabilistic_simulation=simulation.like_simulation(),
    resource_adequacy_metric = ExpectedUnservedEnergy(),
    threshold=0.01
	)
elcc = elcc_inst.evaluate()